In [1]:
import numpy as nm
import pandas as pd
import csv
!pip install mpl_finance
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#f = open('2010_2020_daily_CRSP.csv')
#csv_f = csv.reader(f)
#df = pd.read_csv('/content/drive/MyDrive/Dataroom/2015_2020_OHLCV.csv') # 5 years
#df = pd.read_csv('/content/drive/MyDrive/Dataroom/2000_2020_OHLCV.csv') # 20 years
#df = pd.read_csv('/content/drive/MyDrive/Dataroom/1999_2020_OHLCV.csv') # 20 years with dividends
df = pd.read_csv('/content/drive/MyDrive/Dataroom/1999_2020_shrcd12.csv') # 20 years SHRCD=12
#df = pd.read_csv('/content/drive/MyDrive/Dataroom/1999_2020_ETFs.csv') # 20 years ETFs

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.size

38988375

In [4]:
df_copy = df.copy().fillna(0)

In [5]:
#Only share codes 10, 11, 31 filter
#df_copy = df_copy.loc[df_copy['SHRCD'].isin([ 10.0, 11.0 or 31.0])]

In [6]:
df_copy.size

38988375

In [7]:
#Adjust prices
df_copy['Close']=df_copy['PRC']/df_copy['CFACPR']
df_copy['Open']=df_copy['OPENPRC']/df_copy['CFACPR']
df_copy['High']=df_copy['ASKHI']/df_copy['CFACPR']
df_copy['Low']=df_copy['BIDLO']/df_copy['CFACPR']

df_copy['Vol']=df_copy['VOL']*df_copy['CFACSHR']

df_copy['Dividend']=df_copy['DIVAMT']/df_copy['CFACPR']

In [8]:
#Rename columns
df_copy.rename(columns={'PERMNO': 'Permno',
                        'date': 'Date',
                        'TICKER': 'Ticker',
                        'PRC': 'RealClose'}, inplace=True)


In [9]:
#Drop waste
df_copy.drop({'SHRCD','SHROUT','CFACPR','CFACSHR', 'OPENPRC', 'OPENPRC','ASKHI','BIDLO','VOL','DIVAMT'}, inplace=True, axis=1)

In [10]:
df_copy

,Permno,Date,SICCD,Ticker,PERMCO,RealClose,Close,Open,High,Low,Vol,Dividend
0,10201,07/23/2010,4931,AT,53455,12.4800,12.4800,12.30,12.87,12.1000,143700.0,0.00000
1,10201,07/26/2010,4931,AT,53455,12.6200,12.6200,12.48,12.82,12.4800,129600.0,0.00000
2,10201,07/27/2010,4931,AT,53455,12.7900,12.7900,12.94,12.94,12.6400,111400.0,0.00000
3,10201,07/28/2010,4931,AT,53455,12.4895,12.4895,12.97,12.97,12.4895,103600.0,0.08831
4,10201,07/29/2010,4931,AT,53455,12.4300,12.4300,12.69,12.79,12.4000,104500.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...
2599220,93427,12/24/2020,3661,FN,53445,76.0500,76.0500,76.93,77.36,75.3100,121244.0,0.00000
2599221,93427,12/28/2020,3661,FN,53445,77.8600,77.8600,76.88,78.32,76.6550,119630.0,0.00000
2599222,93427,12/29/2020,3661,FN,53445,75.5500,75.5500,78.26,78.26,75.0000,99347.0,0.00000
2599223,93427,12/30/2020,3661,FN,53445,76.4900,76.4900,75.65,77.13,75.6500,66852.0,0.00000


In [11]:
#Replace NaN with 0
#df_copy['DIVAMT'] = df_copy['Dividend'].fillna(0)


In [12]:
#df_copy.loc[df_copy['Dividend'].isin([0])]

In [13]:
df_test = df_copy.loc[df_copy['Ticker'].isin(['NIO'])]

In [14]:
df_test[(df_test['Dividend'] != 0)]

,Permno,Date,SICCD,Ticker,PERMCO,RealClose,Close,Open,High,Low,Vol,Dividend


In [15]:
df_test

,Permno,Date,SICCD,Ticker,PERMCO,RealClose,Close,Open,High,Low,Vol,Dividend


In [16]:
df_print = df_copy

In [18]:
#df['PERMNO'].unique()
#df['PERMNO'].nunique()
#i=0
for permno in df_print['Permno'].unique():
    sub_df = df_print[df_print['Permno'] == permno]
    all_tickers = sub_df['Ticker'].unique()
    tick = sub_df[sub_df['Date'] == sub_df['Date'].max()]['Ticker'].tolist()[0]
    print(f'Current file we are working on: {permno} ticker: {tick} ')
    if len(all_tickers ) > 1:
      print(f'Current tickers are: ', all_tickers)
   #sub_df.download(f'{permno}_{tick}.csv')
    sub_df.to_csv(f'/content/drive/MyDrive/Individual_data/uus/{tick}.csv', index=False)
   # i=i+1
   # if i==100:
   #   break

Current file we are working on: 10201 ticker: AT 


FileNotFoundError: ignored

In [ ]:
#sub_df['TICKER'].unique()#.item()
sub_df[sub_df['date'] == sub_df['date'].max()]['TICKER'].item()

In [ ]:
sub_df = df[df['PERMNO']==10028]
sub_df.head()
##df[df['PERMNO']==10028]['PRC'].plot()
##df = df[['date', 'open', 'high', 'low', 'close', 'volume']] =

In [ ]:
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates

In [ ]:
plot_df = df_test

#if necessary convert to datetime
plot_df.date = pd.to_datetime(df.date)


plot_df = plot_df[['Date', 'Open', 'High', 'Low', 'Close', 'Vol']]
plot_df["Date"] = plot_df["Date"].apply(mdates.date2num)

plt.figure(figsize=(15, 10), dpi=80)
f1 = plt.subplot2grid((6, 4), (1, 0), rowspan=6, colspan=4)
candlestick_ohlc(f1, plot_df.values, width=.6, colorup='#53c156', colordown='#ff1717')
f1.xaxis_date()
f1.xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d'))

plt.xticks(rotation=45)
plt.ylabel('Stock Price')
plt.xlabel('Date')
plt.show()

In [ ]:
df_copy

In [ ]:
#help(db.get_table)
#db.list_tables(library='crsp')
#db.describe_table(library="crsp", table="stocknames")
#db.describe_table(library="crsp", table='saz_ds_dly')
#stocknames = db.get_table(library='crsp', table="stocknames", columns=['permno','permco','ticker','comnam'], obs=10)
#stocknames.head()
#stocknames = db.get_table(library="crsp", table='saz_ds_dly', columns=['KYPERMNO','CALDT','BIDLO','ASKHI','BID','ASK'], obs=10)
#stocknames = db.get_table(library="crsp", table='saz_ds_dly', obs=10)
#stocknames.head()

In [ ]:
dly1 = db.get_table(library="crsp", table='saz_ds_dly', obs=1000)
dly2 = db.get_table(library="crsp", table='saz_dp_dly',obs=1000)
dly3 = pd.merge(dly1, dly2, on=["kypermno", "caldt"])
dly3.head()

In [ ]:
dly3.drop(columns=['prc','ret','retx','tcap'])

In [ ]:
for row in dly3.itertuples():
    permno = row.kypermno*1
    dly3.to_csv(str(permno)+'.csv', index = False)

In [ ]:
df['kypermno'].unique()

In [ ]:
df=dly3

In [ ]:
for permco in df['kypermno'].unique():
    sub_df = df[df['kypermno'] == permco]
    sub_df.to_csv(f'{permco}.csv', index=False)

In [ ]:
df['prc'].plot()